# **PROYECTO 1 BI - DETECCIÓN DE FAKE NEWS**
---

Integrantes equipo #11:
*   Estudiante #1: Juan Pablo Barón - 202210502
*   Estudiante #2: María José Amorocho - 202220179
*   Estudiante #3: Julian Mondragón - 202221122

---

## Carga de datos

In [ ]:
!pip install num2words
!pip install spacy
!python -m spacy download es_core_news_sm

In [ ]:
from pandas import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re, string, unicodedata
from num2words import num2words
from nltk.corpus import stopwords
import spacy


# Punkt permite separar un texto en frases.
nltk.download('punkt')
#Descargar palabras vacías (stopwords)
nltk.download("stopwords")
stop_words = set(stopwords.words("spanish"))


data_set = pd.read_csv('./Data/fake_news_spanish-2k.csv', delimiter=";")
data_set.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\majoa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\majoa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: './Data/fake_news_spanish-1k.csv'

## **1. Entendimiento del negocio y enfoque analítico**

### 1.1. Generalidades

### 1.2. Impacto y enfoque analítico

---

## **2. Entendimiento y preparación de los datos**

### Selección de variables

Para saber si una noticia es falsa o no, el modelo unicamente usará los datos de 'Titulo' y 'Descripcion' de la noticia. Pro una parte, la columna 'ID' dentro del dataset, no aporta ninguna información y por otro lado, la fecha de una noticia no es relevante para el caso dado que la verdacidad de la noticia no depende del día que haya sido escrita. Se dejará la columna de Label ya que a partir de esta es que el modelo puede aprender

In [2]:
def definir_variables(data_set_inicial):
    features = ['Titulo', 'Descripcion', 'Label']
    return data_set_inicial[features]
    
df = definir_variables(data_set)
df.shape

(1988, 3)

### 2.1. Perfilamiento de datos

Número de datos

In [3]:
data_set.shape

(1988, 5)

Tipos de dato presentes, cantidad de datos nulos y columnas que componen el data set

In [4]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           1988 non-null   object
 1   Label        1988 non-null   int64 
 2   Titulo       1988 non-null   object
 3   Descripcion  1988 non-null   object
 4   Fecha        1988 non-null   object
dtypes: int64(1), object(4)
memory usage: 77.8+ KB


Datos vacíos

In [5]:
df_null = pd.DataFrame(data_set.apply(lambda x: (x == "").sum()), columns=["Vacios"])
print(df_null)

             Vacios
ID                0
Label             0
Titulo            0
Descripcion       0
Fecha             0


Filas duplicadas

In [6]:
duplicated_rows = data_set.loc[data_set.duplicated(keep=False)]
duplicated_rows.shape

(0, 5)

Filas duplicadas parcialmente

In [7]:
def find_partial_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identifica filas con el mismo título y descripción pero diferente label.
    
    Parámetros:
    df (pd.DataFrame): DataFrame de entrada con columnas ['id', 'label', 'Titulo', 'descripcion', 'fecha'].
    
    Retorna:
    pd.DataFrame: DataFrame con duplicados parciales.
    """
    # Identificar duplicados basados en 'Titulo' y 'descripcion'
    duplicated_mask = df.duplicated(subset=['Titulo', 'Descripcion'], keep=False)
    
    # Filtrar las filas que cumplen la condición
    duplicates = df[duplicated_mask].sort_values(by=['Titulo', 'Descripcion'])
    
    # Agrupar por título y descripción, asegurando que hay más de un label diferente
    duplicates_grouped = duplicates.groupby(['Titulo', 'Descripcion'])['Label'].nunique().reset_index()
    
    # Filtrar los casos donde hay más de un label único
    filtered_duplicates = duplicates_grouped[duplicates_grouped['Label'] > 1]

    # Unir con el DataFrame original para obtener los duplicados completos
    result = df.merge(filtered_duplicates[['Titulo', 'Descripcion']], on=['Titulo', 'Descripcion'])

    return result

df = find_partial_duplicates(data_set)
df.shape


(0, 5)

### 2.2. Preparación de datos

#### Limpieza de datos

Eliminar filas duplicadas

In [8]:
def remove_duplicates(df: pd.DataFrame):
    """
    Elimina las filas duplicadas del dataSet.
    
    Parámetros:
    df (pd.DataFrame): DataFrame de entrada.
    
    Retorna:
    pd.DataFrame: DataFrame sin filas duplicadas.
    """
    return df.drop_duplicates().reset_index(drop=True)

df = remove_duplicates(data_set)
df.shape

(1988, 5)

Eliminar filas duplicadas parcialmente (noticias con un mismo titular y descripción, pero diferente label)

In [9]:
def eliminar_duplicados_parciales(df):
    """
    Elimina filas donde 'Titulo' y 'Descripcion' sean iguales, pero 'Label' sea diferente.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con las columnas 'Titulo', 'Descripcion' y 'Label'.
    
    Retorna:
    pd.DataFrame: DataFrame sin los duplicados parciales.
    """
    # Contar cuántos valores únicos de Label existen por cada combinación de Titulo y Descripcion
    conteo_labels = df.groupby(['Titulo', 'Descripcion'])['Label'].nunique()
    
    # Identificar las combinaciones que tienen más de un Label distinto (es decir, duplicados parciales)
    duplicados_parciales = conteo_labels[conteo_labels > 1].index
    
    # Filtrar el DataFrame eliminando estas combinaciones
    df_filtrado = df[~df.set_index(['Titulo', 'Descripcion']).index.isin(duplicados_parciales)]
    
    return df_filtrado.reset_index(drop=True)

df = eliminar_duplicados_parciales(data_set)
df.shape


(1988, 5)

A continuación, se muestra un método para hacer la limpieza completa de los datos

In [10]:
def limpiar_data(data_set):
    df_variables = definir_variables(data_set)
    df_duplicados = remove_duplicates(df_variables)
    df_limpio = eliminar_duplicados_parciales(df_duplicados)
    return df_limpio

df = limpiar_data(data_set)
df.shape

(1988, 3)

#### Eliminación de ruido

Funciones para dejar la información en texto plano (remover caracteres no ASCII, convertir las palabras en minúscula, remover la puntuación, reemplazar los números y remover stopwords)

In [11]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for w in words:
        new_w = w.lower()
        new_words.append(new_w)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang="es")
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words


#### Tokenización, lematización y normalización

Para procesar los títulos de las noticias y su descripción se procede a ejecutar los siguietes pasos:
1. **Tokenización**: Dividir frases u oraciones en palabras con el fin de desglozar las palabras correctamente para el posterior análisis

2. **Lematización**: Cada palabra se reduce a su forma base o lema

3. **Normalización**: Se hace uso de la función de reducción de ruido para colocar cada palaba en minuscula, sin tildes, remover los signos de puntuación, caracteres no ASCII, reemplazar los números y las palabras vacías

In [12]:
# Cargar modelo spaCy
nlp = spacy.load("es_core_news_sm")

# Función optimizada para tokenizar varias columnas
def tokenizar_columnas(df, columnas):
    """
    Tokeniza varias columnas en un DataFrame usando spaCy.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame con los datos.
    - columnas (list): Lista de nombres de columnas a tokenizar.

    Retorna:
    - DataFrame con nuevas columnas de tokens.
    """
    # Convertir las columnas a string y reemplazar NaN por ""
    for col in columnas:
        df[col] = df[col].astype(str).fillna("")

    # Aplicar `nlp.pipe()` en paralelo para mayor velocidad
    for col in columnas:
        df[f"{col}_tokens"] = [
            [token.text for token in doc if not token.is_space] 
            for doc in nlp.pipe(df[col], batch_size=50)
        ]
    
    return df

def procesar_texto(df, columnas):
    """
    Tokeniza, lematiza y aplica preprocesamiento a múltiples columnas de un DataFrame con spaCy.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame con los datos.
    - columnas (list): Lista de nombres de columnas a procesar.

    Retorna:
    - Nuevo DataFrame con solo las columnas tokenizadas, lematizadas y procesadas, con 'label' como primera columna.
    """
    # Convertir las columnas a string y reemplazar NaN por ""
    for col in columnas:
        df[col] = df[col].astype(str).fillna("")

    # Aplicar `nlp.pipe()` una sola vez por columna para eficiencia
    datos_procesados = {}
    
    for col in columnas:
        resultados = [
            ([token.text for token in doc if not token.is_space],  # Tokens
             [token.lemma_ for token in doc if not token.is_space])  # Lemas
            for doc in nlp.pipe(df[col], batch_size=50)
        ]

        # Separar tokens y lemas en listas
        df[f"{col}_tokens"], df[f"{col}_lemmas"] = zip(*resultados)

        # Aplicar la función `preprocessing` sobre los tokens
        datos_procesados[f"{col}_tokens_clean"] = df[f"{col}_tokens"].apply(preprocessing)

    # Crear nuevo DataFrame solo con las columnas procesadas + label
    nuevo_df = pd.DataFrame(datos_procesados)

    # Agregar la columna label como la primera
    nuevo_df.insert(0, "Label", df["Label"])  

    return nuevo_df


# Aplicar tokenización
data_setC = limpiar_data(data_set)
print("data limpia")
df = procesar_texto(data_setC, ["Titulo", "Descripcion"])
df.head(10)



data limpia


,Label,Titulo_tokens_clean,Descripcion_tokens_clean
0,1,"[the, guardian, va, sanchez, europa, necesita,...","[diario, britanico, publico, pasado, jueves, e..."
1,0,"[revelan, gobierno, negocio, liberacion, mirel...","[revelan, gobierno, negocio, liberacion, mirel..."
2,1,"[ahora, nunca, joan, fuster, estatuto, valenci...","[valencianismo, convoca, castello, fiesta, gra..."
3,1,"[iglesias, alienta, yolanda, diaz, erc, eh, bi...","[politica, igual, negociar, empresarios, negoc..."
4,0,"[puigdemont, seria, ninguna, tragedia, repetic...","[entrevista, punt, avui, lider, jxcat, desdram..."
5,1,"[pnv, consolida, mayoria, pse, salva, papeles,...","[nacionalistas, consiguen, alcaldias, bilbao, ..."
6,0,"[exconsejero, nuria, marin, pide, indulto, cas...","[familiares, aluden, honestidad, integridad, p..."
7,1,"[fiscalia, pide, prision, incondicional, siete...","[suprime, delito, rebelion, imputo, inicialmen..."
8,1,"[jose, manuel, perez, tornero, creador, televi...","[futuro, presidente, rtve, licenciado, ciencia..."
9,0,"[ayusizacion, bng, santiago, abascal, instruye...","[pablo, santiago, abascal, planea, vivir, rent..."


#### Vectorización

---

## **3. Modelado y evaluación**

### 3.1. Estudiante 1 (Juan Pablo Barón): Modelo 1

### 3.2. Estudiante 2 (María José Amorocho): Modelo 2

### 3.3. Estudiante 3 (Julian Mondragón): Modelo 3

## **4. Resultados**

### 4.1. Descripción de resultados obtenidos y cómo aportan a lograr el objetivo del modelo planteado

### 4.2. Estrategia

### 4.3 Archivo de predicciones sobre los datos de prueba en formato CSV